![](ubc_header.png)

# Climate Friendly Food System (CFFS) Labelling Pilot
***

## Objective

Implement the Climate-Friendly and Just Food System (CFFS &Just) definition at the UBC Campus by producing the weighted metric that informs the choice of icon for each menu item.

## Data Source

**Recipes Data**: extracted from UBC culinary services management platform Optimum Control
    
    Items.xml: 
    Ingredients.xml:
    Preps.xml:
    Products.xml:
    Conversions.xml:

**cool_food_ghg.csv**: extracted from the [Cool Food Calculator](https://coolfood.org/pledge/). For this project, we use emission factors for the active total supply chain emissions in the North America region. The food category is at the detailed level for higher results accuracy

**ingredient_list.csv**: all ingredients appeared in the top 10 selling menu items with their associated food category in the Cool Food Category

## Assumptions

* The same GHG emission factor will be assigned to different forms (puree, sliced, chopped etc) of the same raw ingredient

* We assume the GHG factors for different varieties of one ingredient are the same (i.e Red/Yellow Onion)

* The loss/add of ingredient weight during the cooking process will be ignored

* The amount of unusable part and wastes during ingredients processing process are ignored 

* Ignore GHG emissions from the cooking process

* Assume the GHG emission factor for water is zero, and ignore the water use in the cooking process


## Import Libraries

In [1]:
#!pip3 install pdpipe

In [2]:
import numpy as np 
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt

import csv
from itertools import islice

In [3]:
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse

## Load Data

#### Load Recipe Data

In [4]:
# Load Items.xml
xtree = et.parse('/Users/silvia/cffs-label/data/oc/Items.xml')
xroot = xtree.getroot()

ItemId = []
Description = []
Cost = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []

for item in xtree.iterfind('Item'):
    ItemId.append(item.attrib['id'])
    Description.append(item.findtext('Description'))
    Cost.append(item.findtext('Cost'))
    CaseQty.append(item.findtext('CaseQty'))
    CaseUOM.append(item.findtext('CaseUOM'))
    PakQty.append(item.findtext('PakQty'))
    PakUOM.append(item.findtext('PakUOM'))
    InventoryGroup.append(item.findtext('InventoryGroup'))
    
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'Cost': Cost, 'CaseQty': CaseQty, 'CaseUOM': CaseUOM,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup})
Items

,ItemId,Description,Cost,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4472,AVOCADO MX,1.2450,20.000,CT,1.000,CT,PRODUCE
1,I-3133,BASE CHICK CONC LIQ G/FREE,0.0148,4.000,pak,946.000,ml,FOOD - GROCERY
2,I-3141,BASE VEG CONC LIQ G/FREE,0.0131,4.000,pak,946.000,ml,FOOD - GROCERY
3,I-3619,BAY LEAF WHL SHAKER TFC,0.0536,8.000,each,84.000,g,SPICES
4,I-4516,BEANS GREEN(25 LBS),1.4200,1.000,lb,1.000,lb,PRODUCE
...,...,...,...,...,...,...,...,...
66,I-9341,"TORTILLA FLOUR 6""",0.1185,24.000,pak,12.000,CT,BAKED GOODS
67,I-4489,TURNIP BABY WHITE(24CT) BC,2.4000,1.000,BUNCH,1.000,BUNCH,PRODUCE
68,I-3695,VINEGAR RED WINE TFC,1.9920,2.000,pail,5.000,L,FOOD - GROCERY
69,I-5983,WATER FOR RECIPES,0,1.000,L,1.000,L,BEVERAGE


In [5]:
# Load Ingredients.xml
xtree = et.parse('/Users/silvia/cffs-label/data/oc/Ingredients.xml')
xroot = xtree.getroot()

IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

for x in xtree.iterfind('Ingredient'):
    IngredientId.append(x.attrib['ingredient'])
    Conversion.append(x.attrib['conversion'])
    InvFactor.append(x.attrib['invFactor'])
    Qty.append(x.attrib['qty'])
    Recipe.append(x.attrib['recipe'])
    Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Conversion': Conversion, 
                      'InvFactor': InvFactor, 'Qty': Qty, 'Recipe': Recipe,'Uom': Uom})
Ingredients

,IngredientId,Conversion,InvFactor,Qty,Recipe,Uom
0,I-4598,1.00000000,0.0013,1.000,P-12954,CT
1,I-4793,2.20462000,1.2048,10.000,P-18746,Kg
2,I-3643,0.00100000,0.1837,225.000,P-18907,g
3,I-6026,1.00000000,0.8163,1000.000,P-18907,g
4,I-4381,1.00000000,1.1111,1.000,P-24452,lb
...,...,...,...,...,...,...
168,P-26225,0.00220462,1.0000,135.000,R-57711,g
169,P-26234,1.00000000,1.0000,1.000,R-57711,ea
170,P-50429,1.00000000,1.0000,10.000,R-57711,g
171,P-50497,0.00100000,1.0000,180.000,R-57711,g


In [6]:
# Load Preps.xml
xtree = et.parse('/Users/silvia/cffs-label/data/oc/Preps.xml')
xroot = xtree.getroot()

PrepId = []
Description = []
Cost = []
PakQty = []
PakUOM = []
InventoryGroup = []

for x in xtree.iterfind('Prep'):
    PrepId.append(x.attrib['id'])
    Description.append(x.findtext('Description'))
    Cost.append(x.findtext('Cost'))
    PakQty.append(x.findtext('PakQty'))
    PakUOM.append(x.findtext('PakUOM'))
    InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description, 'Cost': Cost,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup})
Preps

,PrepId,Description,Cost,PakQty,PakUOM,InventoryGroup
0,P-26234,BATCH|Roasted Garlic Bread,0.6433,16.000,ea,PREP
1,P-26231,BUTTER|Roasted Garlic,0.0131,550.000,g,PREP
2,P-50500,CHIFFONADE|Basil,0.0354,190.000,g,
3,P-24750,CHOPPED|Cilantro,19.1000,0.500,Kg,
4,P-26913,COOKED|Baked Acon Squash|SL,7.8758,1.000,Kg,
5,P-56836,COOKED|Chicken Marbella,8.4356,3.500,Kg,
6,P-26143,COOKED|Rice|Brown Basmati,2.4416,4.536,Kg,PREP
7,P-32718,COOKED|Rice|Chicken Fried,5.6078,7.000,Kg,PREP
8,P-34121,COOKED|StirFryChicken,11.6436,4.000,Kg,PREP
9,P-41094,"DICED|Carrot|1/4""",3.7530,11.170,Kg,


In [7]:
# Load Products.xml
xtree = et.parse('/Users/silvia/cffs-label/data/oc/Products.xml')
xroot = xtree.getroot()

ProdId = []
Description = []
Cost = []

for x in xtree.iterfind('Prod'):
    ProdId.append(x.attrib['id'])
    Description.append(x.findtext('Description'))
    Cost.append(x.findtext('Cost'))
    
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'Cost': Cost})

Products

,ProdId,Description,Cost
0,R-55298,POP|ChickenMole Taco|LM,0.5404
1,R-55284,SQR|Charro Beans Side|LM,0.3631
2,R-51313,SQR|ChayoteSaladTostadas,1.2176
3,R-54176,SQR|Chicken Cacciatore,2.4857
4,R-57710,SQR|Chicken Cacciatore +1,3.3982
5,R-57711,SQR|Chicken Cacciatore +2,4.2684
6,R-47146,SQR|Chicken Fried Rice,0.9533
7,R-32868,SQR|Chicken Fried Rice 170g,0.9533
8,R-56837,SQR|Chicken Marbella,1.3023
9,R-57332,SQR|Chicken Marbella+1,1.7026


In [8]:
# Load Conversions.xml
xtree = et.parse('/Users/silvia/cffs-label/data/oc/Conversions.xml')
xroot = xtree.getroot()

ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

for x in xtree.iterfind('Conversion'):
    ConversionId.append(x.attrib['id'])
    Multiplier.append(x.attrib['multiplier'])
    ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
    ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
    ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
    ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom})
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L
...,...,...,...,...,...,...
89,I-6443,0.00406504,1.0000,cup,246.0000,g
90,I-7060,0.03401361,1.0000,fl oz,29.4000,g
91,I-8060,0.07042254,1.0000,Tbsp,14.2000,g
92,I-8060,0.00440529,1.0000,cup,227.0000,g


#### Load Emission Factors

In [9]:
cfc_filename = '/Users/silvia/cffs-label/data/raw/cool_food_ghg.csv'
cfc = pd.read_csv(cfc_filename)
cfc.head()

,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,beef & buffalo meat,41.3463
1,lamb/mutton & goat meat,41.6211
2,pork (pig meat),9.8315
3,"poultry (chicken, turkey)",4.3996
4,butter,11.4316


In [10]:
cfc.shape

(54, 2)

#### Load Ingredient List

In [11]:
in10_filename = '/Users/silvia/cffs-label/data/raw/ingredient_list.csv'
in10 = pd.read_csv(in10_filename)
in10.head()

,Name of Ingredient,Food Category in the Cool Food Pledge
0,Black Beans|Drained,beans and pulses (dried)
1,Butter|Unsalted,butter
2,Arugula,cabbages and other brassicas (broccoli)
3,Cabbage,cabbages and other brassicas (broccoli)
4,Cauliflower Florets,cabbages and other brassicas (broccoli)


In [12]:
in10.shape

(116, 2)

## Data Cleaning

#### Construct Dictionary for Food Categories and Emission Factors

In [13]:
def row_csvtodict(csv_file):
    dict_club={}
    with open(csv_file)as f:
        reader = csv.reader(f,delimiter=',', )
        headers = next(reader)
        for row in reader:
            dict_club[row[0]]=row[1]
    return dict_club

row_csvtodict(cfc_filename)

{'beef & buffalo meat': '41.3463',
 'lamb/mutton & goat meat': '41.6211',
 'pork (pig meat)': '9.8315',
 'poultry (chicken, turkey)': '4.3996',
 'butter': '11.4316',
 'cheese': '8.9104',
 'ice cream': '4.0163',
 'cream': '6.9824',
 "milk (cow's milk)": '2.2325',
 'yogurt': '2.9782',
 'eggs': '3.6615',
 'fish (finfish)': '4.9798',
 'crustaceans (shrimp/prawns)': '21.1274',
 'mollusks': '2.4351',
 'animal fats': '6.9693',
 'other legumes': '1.6042',
 'beans and pulses (dried)': '1.6776',
 'peas': '0.6995',
 'peanuts/groundnuts': '1.692',
 'soybeans/tofu': '1.7542',
 'other grains/cereals': '1.4785',
 'corn (maize) ': '0.9734',
 'oats (oatmeal)': '2.3017',
 'wheat/rye (bread, pasta, baked goods)': '1.5225',
 'rice': '2.5345',
 'tree nuts and seeds': '4.2854',
 'almond milk': '0.7021',
 'oat milk': '0.9943',
 'rice milk': '0.6972',
 'soy milk': '0.489',
 'other fruits': '0.4306',
 'apples': '0.3581',
 'bananas': '0.7115',
 'berries': '1.6547',
 'citrus fruit ': '0.3942',
 'cabbages and oth

## Data Analysis

#### Mapping Ingredients to GHG Factors (For the top 10 menu items)

In [14]:
import openpyxl

df1 = pd.DataFrame(in10)
df2 = pd.DataFrame(cfc)

In [15]:
df1

,Name of Ingredient,Food Category in the Cool Food Pledge
0,Black Beans|Drained,beans and pulses (dried)
1,Butter|Unsalted,butter
2,Arugula,cabbages and other brassicas (broccoli)
3,Cabbage,cabbages and other brassicas (broccoli)
4,Cauliflower Florets,cabbages and other brassicas (broccoli)
...,...,...
111,Sauce|Chili Hot Sriracha,NaN
112,Sauce|Sweet & Sour,NaN
113,Sauce|Sweet Chili,NaN
114,Sauce|Worchestershire,NaN


In [16]:
df2

,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,beef & buffalo meat,41.3463
1,lamb/mutton & goat meat,41.6211
2,pork (pig meat),9.8315
3,"poultry (chicken, turkey)",4.3996
4,butter,11.4316
5,cheese,8.9104
6,ice cream,4.0163
7,cream,6.9824
8,milk (cow's milk),2.2325
9,yogurt,2.9782


In [17]:
result = pd.merge(df1, df2.loc[:,['Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'Food Category in the Cool Food Pledge', 
                  right_on = 'Food Category')
result

,Name of Ingredient,Food Category in the Cool Food Pledge,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,Black Beans|Drained,beans and pulses (dried),beans and pulses (dried),1.6776
1,Butter|Unsalted,butter,butter,11.4316
2,Arugula,cabbages and other brassicas (broccoli),cabbages and other brassicas (broccoli),0.6220
3,Cabbage,cabbages and other brassicas (broccoli),cabbages and other brassicas (broccoli),0.6220
4,Cauliflower Florets,cabbages and other brassicas (broccoli),cabbages and other brassicas (broccoli),0.6220
...,...,...,...,...
111,Sauce|Chili Hot Sriracha,NaN,NaN,NaN
112,Sauce|Sweet & Sour,NaN,NaN,NaN
113,Sauce|Sweet Chili,NaN,NaN,NaN
114,Sauce|Worchestershire,NaN,NaN,NaN


#### Calculate GHG Emissions